In [ ]:
import numpy as np
from numpy import array, sin, cos, dot, random, zeros, ones, exp
from scipy.optimize import minimize, root
from scipy.linalg import solve, norm
from math import pi
import matplotlib.pyplot as plt
from matplotlib import animation,rc
%matplotlib inline
from IPython.display import HTML

In [ ]:
def acrobot_gradient(x, u):
    friction = 2
    g = 9.8
    M = array([[3 + 2*cos(x[1]), 1+cos(x[1])],[1+cos(x[1]), 1]])
    c1 = x[3]*(2*x[2]+x[3])*sin(x[1]) + 2*g*sin(x[0]) + g*sin(x[0]+x[1])
    c2 = -x[2]**2*sin(x[1]) + g*sin(x[0]+x[1])
    a = np.concatenate([x[2:4], solve(M,array([c1-friction*x[2], c2-friction*x[3]]))])
    B = np.concatenate([[0,0],solve(M,array([0,1]))])
    xd = a + B*u
    return xd
#r=k=1
#cost_function = lambda x,u,r,k : r/2*u**2 + 1 - exp(k*cos(x[0]) + k*cos(x[1])-2*k)
#cost_function = lambda x,u : u**2 + dot(x,x)

In [ ]:
starting_point = array([0.001,0,0,0])
def acrobot_next_step(x,u):
    
    xd = acrobot_gradient(x,u)
    new_x = x + xd*dt
    #limit angle range to [0,2 pi]
    new_x[0]%=(2*pi)
    new_x[1]%=(2*pi)
    #avoid acceleration overflow
    return np.clip(new_x,-100,100)

def grad_lagrangian_trick(u):
    # u is a vector of u0 to uN-1
    n=len(u)
    x = zeros([n+1,4])
    x[0]=starting_point
    lam = zeros([n,4])
    lam[n-1] = zeros(4)
    dldu = zeros(n)
    
    #define partial derivation functions
    def dgdx(x,u):
        k=1
        out = zeros(4)
        x1= x[0]
        x2= x[1]
        out[0] = exp(k*cos(x1)+k*cos(x1)-2*k)*k*sin(x1)
        out[1] = exp(k*cos(x2)+k*cos(x2)-2*k)*k*sin(x2)
        return out
    dgdu = lambda x,u : 0
    
    delta = 0.0001
    def dfdxT(x,u):
        out = zeros([4,4])
        for i in range(4):
            delta_v = zeros(4)
            delta_v[i] = delta
            #print(x+delta_v)
            out[i] = ((acrobot_gradient(x+delta_v,u)-acrobot_gradient(x-delta_v,u))*dt+2*delta_v)/2/delta
        return out
    dfdu = lambda x,u : ((acrobot_gradient(x,u+delta)-acrobot_gradient(x,u-delta))*dt+2*delta)/2/delta
    
    #calculate gradient through back propagation
    for i in range(n):
        x[i+1]=acrobot_next_step(x[i],u[i])
        
    for i in reversed(range(1,n)):
        lam[i-1]=dgdx(x[i],u[i])+dot(dfdxT(x[i],u[i]),lam[i])
    
    for i in range(n):
        dldu[i]=dgdu(x[i],u[i])+dot(lam[i],dfdu(x[i],u[i]))
    
    return dldu

#change here
#------------
Duration = 1;
FPS = 100;
#------------
dt = 1./FPS
N = int(Duration*FPS);

u = zeros(N)#random.normal(0,size=N)

out = root(grad_lagrangian_trick,u)

## Simulation

In [ ]:

u = out.x
x = np.zeros((N, 4))
x[0] = starting_point
for i in range(N-1):
    x[i+1] = x[i] + dt*acrobot_gradient(x[i], u[i])
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( -2.2, 2.2))
ax.set_ylim((-2.2, 2.2))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(k):
    x1 = cos(x[k,0]+pi/2);
    y1 = sin(x[k,0]+pi/2);
    x2 = x1 + cos(x[k,0]+x[k,1]+pi/2);
    y2 = y1 + sin(x[k,0]+x[k,1]+pi/2);
    
    xs = [0, x1, x2]
    ys = [0, y1, y2]
    line.set_data(xs, ys)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, blit=True)


In [ ]:
HTML(anim.to_html5_video())

# test codes, will delete in the end

In [ ]:
#brute force method, not converging

def lagrangian(input_array):
    #x is x1 to xN,followed by U0 to UN-1, followed by lambda0 to lambdaN-1
    x = np.concatenate([[pi,0,0,0],input_array[:(n-1)*4]])
    u = input_array[(n-1)*4:(n-1)*4+n]
    lam = input_array[(n-1)*4+n:-1]

    L=0
    for i in range(n-1):
        #current x
        xx = x[i*4:(i+1)*4]
        #next x
        xx_p1 = x[(i+1)*4:(i+2)*4]
        #difference between x+dt*dx and x_next
        delta = xx+dt*acrobot_motion(xx,u[i])-xx_p1
        #minimize energy
        cost = cost_function(xx,u[i])
        lagragian_term = lam[i]*(delta[0:1]%(2*pi))
        L+=cost+lagragian_term
    return L
#init = np.ones(n*4+n+n)
#out = minimize(lagrangian,init)

In [ ]:
x[:,0]

In [ ]:
out